## Previsão de Falha

Modelo para a predição da probabilidade de falha de dispositivos, a fim de reduzir custos ao antecipar o seu reparo.

Para isso, após tratamento dos dados, foi gerado um modelo de floresta randômica para esta predição. Através de seus resultados, obtemos a resposta para a seguinte pergunta: Podemos realmente reduzir custos com este modelo de ML?

Resposta: Sim. O modelo gera aproximadamente 58,7% de saving no conjunto de teste estudado, em comparação à situação na qual não ocorre a predição da falha dos dispositivos.

#### 1 - Importação dos Dados

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
import joblib

In [2]:
df_raw = pd.read_csv('data/full_devices.csv', sep=',', encoding='latin')
df_raw.head(5)

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


#### 2 - Pré-processamento dos Dados

Verifica-se campos faltantes ou a presença de linhas duplicadas. Além disso, é realizado um rebalanceamento das classes, de forma a torna-las mais homogêneas.

In [3]:
df = df_raw.copy()
df.isna().sum()

date          0
device        0
failure       0
attribute1    0
attribute2    0
attribute3    0
attribute4    0
attribute5    0
attribute6    0
attribute7    0
attribute8    0
attribute9    0
dtype: int64

In [4]:
df.loc[df.duplicated()]

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
101335,2015-07-10,S1F0R4Q8,0,192721392,0,0,0,8,213700,0,0,0


In [5]:
df = df.drop_duplicates()
len(df.loc[df.duplicated()])

0

In [6]:
X = df.drop(['date','device', 'failure'], axis=1)
y = df['failure']

In [7]:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X,y)
print(len(X_resampled))
print(len(y_resampled))

210
210


In [8]:
y_resampled.value_counts()

0    105
1    105
Name: failure, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

#### 3 - Treinamento do Modelo

Dentre os modelos, foi escolhido um modelo de ensemble do tipo Floresta Randômica

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [12]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]


#### 4 - Análise dos Resultados

Foram medidas a acurácia, o F1-Score e a matriz de confusão para determinar não somente a eficácia do modelo, mas também a sua viabilização para o negócio de acordo com o problema apresentado (1.0 para falhas e 0.5 para reparos)

In [13]:
accuracy = accuracy_score(y_test, y_pred)
classif_report = classification_report(y_test, y_pred)

print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:\n', classif_report)

Acurácia: 0.7380952380952381
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.65      0.89      0.76        19
           1       0.88      0.61      0.72        23

    accuracy                           0.74        42
   macro avg       0.76      0.75      0.74        42
weighted avg       0.77      0.74      0.73        42



In [14]:
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

array([[17,  2],
       [ 9, 14]], dtype=int64)

Falsos positivos e Verdadeiros negativos foram considerados como Reparos, enquanto que falsos negativos foram tomados como Falhas

In [15]:
fp = confusion_matrix[1,0] #Reparo: R$0.5
tn = confusion_matrix[1,1] #Reparo: R$0.5
fn = confusion_matrix[0,1] #Falha: R$1.0

In [16]:
preco_original = y_test.to_list().count(1)
preco_predicao = fp * 0.5 + tn * 0.5 + fn * 1

print(f'Preço total dos dispositivos com falha: R${round(preco_original,2)}')
print(f'Preço dos dispositivos após reparo via predição do modelo: R${round(preco_predicao, 2)}')
print(f'Porcentagem Salva: {round(preco_predicao/preco_original*100,2)}%')

Preço total dos dispositivos com falha: R$23
Preço dos dispositivos após reparo via predição do modelo: R$13.5
Porcentagem Salva: 58.7%


#### 6 - Exportação do Modelo

Modelo foi serializado para aplicação futura

In [17]:
trained_model = joblib.dump(model,'models/failure_devices_model.pkl')
trained_scaler = joblib.dump(scaler,'models/failure_devices_scaler.pkl')

#### 7 - Utilização do Modelo

Exemplo de aplicação do modelo utilizando 2 dispositivos extraídos do dataset (e que não foram utilizados no treino ou no teste), para fins de demonstração

In [18]:
def prepare_data(path):
    df_test = pd.read_csv(path, sep=',', encoding='latin')
    data = df_test.drop(['date', 'device', 'failure'], axis=1)
    return df_test, data

def predict_failure(scaler, model, data):

    if len(data) == 1:
        data = data.reshape(1, -1)

    dados_producao_normalizados = scaler.transform(data)
    probabilidades_falha = model.predict_proba(dados_producao_normalizados)[:, 1]

    return probabilidades_falha

In [19]:
load_scaler = joblib.load('models/failure_devices_scaler.pkl')
load_model = joblib.load('models/failure_devices_model.pkl')

df_test = pd.read_csv('data/example_devices.csv', sep=',', encoding='latin')
data = df_test.drop(['date', 'device', 'failure'], axis=1)

failures_prob = predict_failure(load_scaler, load_model, data)
failures = pd.DataFrame({'Prob. Failures (%)': failures_prob})

df_report = pd.concat([df_test,failures], axis=1)
df_report

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,Prob. Failures (%)
0,2015-10-26,W1F0SJJ2,0,103314248,0,0,0,11,355019,0,0,0,0.09
1,2015-10-26,W1F0T0B1,1,95073232,0,0,7,9,354861,22,22,0,0.99


In [20]:
df_report.to_excel('output/failure_report.xlsx', index=False)

#### 8 - Trabalhos Futuros

A otimização dos hiperparâmetros do modelo, a utilização de validação cruzada, a criação de um conjunto de validação e o estudo das correlações dos atributos utilizados são ações que podem ser realizadas para aprimorar os resultados obtidos.

 A adaptação do código, como a obtenção de logs de treinamento e de predição e o tratamento das Exceptions que podem vir a ocorrer, bem como a modelagem para uma API e conteinização, permitem a sua aplicação em produção para ser inserida localmente ou em Cloud.